In [4]:
import os
import matplotlib.pyplot as plt
import pims
import sys
import numpy as np
sys.path.insert(0,'/home/jmamede/scripts')
# from support_pla import othercolor, cell_mask, multiply, rebin
from support_pla import cellpose_mask,cellpose_model
import pandas as pd
from nd2reader import ND2Reader
import glob
import csv
plot = True

In [5]:
# ficheiro = "/run/media/jmamede/Joao/96wells/20201119_143803_601/WellA01_ChannelGFP,DAPI_Seq0000.nd2"
dirname = '/home/jmamede/Data/tests/Well*.nd2'
filelist = glob.glob(dirname)
filelist.sort()


# reader = ND2Reader(ficheiro)
# print(reader.sizes)

# metadata = reader.metadata
# reader.bundle_axes = 'cyx'
# reader.iter_axes = 'z'
# #there is a bug, it reads each visit point as a Z.
# reader.default_coords['v'] = 0
# #reader.sizes['z']

# def plots(reader,cells_masks,nuclei_masks,base,series):
#     %matplotlib inline
#     from cellpose.utils import masks_to_outlines
#     f, axarr = plt.subplots(3)
#     print(axarr.shape)
#     axarr[0].imshow(reader[series][1],vmin=np.min(reader[5][1]),vmax=np.max(reader[5][1])/2
# #         ,cmap="gray"
#                      )
#     axarr[0].imshow(masks_to_outlines(cells_masks),alpha=0.6,cmap='gray'
#         )
# #     axarr[0,2].imshow(reader[i][0],vmin=100,vmax=np.max(reader[5][0])/3
# #         )
#     axarr[2].imshow(reader[series][1],vmin=np.min(reader[5][1]),vmax=np.max(reader[5][1])/2
# #         ,cmap="gray"
#                      )
#     axarr[2].imshow(cells_masks,alpha=0.4,cmap='jet')
    
#     axarr[1].imshow(reader[series][0],vmin=np.min(reader[5][0]),vmax=np.max(reader[5][0])/2
# #                       ,cmap="gray"
#         )
#     axarr[1].imshow(masks_to_outlines(nuclei_masks),alpha=0.4,cmap='gray')
        
        
# #     axarr[1,2].imshow(reader[i][1],vmin=100,vmax=np.max(reader[5][0])/3
# #         )
# #     axarr[1,2].imshow(nuclei_masks
# # #                 ,alpha=0.6
# #         )
#     plt.savefig(base+'v'+str(series),dpi=600)
#     return f,axarr




In [6]:
import napari
from napari.qt import thread_worker
from cellpose.utils import masks_to_outlines
from napari.qt import thread_worker
sys.path.insert(0,'/home/jmamede/scripts')
from support_pla import cellpose_mask,cellpose_model


with napari.gui_qt():
    v = napari.Viewer(show=False)
    v.show()
    #     layer = viewer.add_image(np.random.random((512, 512)))
    def updating(result):
    #         v.layers.remove('image')
        from cellpose.utils import masks_to_outlines
        import time
        [v.layers.pop(0) for la in range(len(v.layers))]    

        v.add_image(
            result[1],
            blending='additive',
            contrast_limits=[np.min(result[1]),np.max(result[1])],
    #         ndisplay=2,
            colormap='blue',
            name='nuclei'
        )   

        v.add_image(
            result[0],
            blending='additive',
            colormap='green',
            name='cells')

        v.add_labels(
        result[2],
        opacity=0.4,
        blending='additive',
            contrast_limits=[np.min(result[2]),np.max(result[2])],
        name='nuclei'
        ) 

        v.add_labels(
        masks_to_outlines(result[3]),
    #     utils.outlines_list(result[0]),
        opacity=0.4,
        blending='additive',
    #         contrast_limits=[np.min(result[1]),np.max(result[1])],
        name='nuclei'
        )
        
    @thread_worker(connect={'yielded': updating})
    def poseri(filelist,chan_cells,chan_nuclei,model,model_nuc):
        import sys, contextlib, io
        import cellpose
        sys.path.insert(0,'/home/jmamede/scripts')
        from support_pla import cellpose_mask,cellpose_model
        from cellpose import io as iocp
        from cellpose import plot,utils
        import matplotlib.pyplot as plt
        import pims
        import time
        import numpy as np
        from nd2reader import ND2Reader
        sys.path.insert(0,'/home/jmamede/scripts')
        # from support_pla import othercolor, cell_mask, multiply, rebin
        from support_pla import cellpose_mask,cellpose_model

        for ficheiro in filelist:
            reader = ND2Reader(ficheiro)
        #     metadata = reader.metadata
            reader.bundle_axes = 'cyx'
            reader.iter_axes = 'z'
            #there is a bug, it reads each visit point as a Z.
            reader.default_coords['v'] = 0
            ncells = 0
            nnuclei = 0

            for i in range(reader.sizes['z']):
    #         for i in [0]:
                #shut cellpose messages up.
    #             with contextlib.redirect_stdout(io.StringIO()):

                cells_masks = cellpose_mask(reader[i][chan_cells],model
                #                             ,size=2500
                                            ,flow_threshold=0.8,size=0,diam=50,cell_prob=-1
                                           )
                nuclei_masks = cellpose_mask(reader[i][chan_nuclei],model_nuc
                #                             ,size=2500
                                            ,flow_threshold=1.2,size=0,diam=25,cell_prob=0
                                           )
        #             ncells += cells_masks.max()
        #             nnuclei += nuclei_masks.max()
                base = os.path.splitext(ficheiro)[0]
                outlines = utils.outlines_list(cells_masks)
                cell_outlines = utils.masks_to_outlines(cells_masks)
                iocp.outlines_to_text(base+'v'+str(i), outlines)

        #             f,axarr =  plots(reader,cells_masks,nuclei_masks,base,i)

                text = "Image#: {0} Number of Nuclei: {1} Number of positive cells: {2} Percentage: {3}".format(i,nuclei_masks.max(),cells_masks.max(),cells_masks.max()/nuclei_masks.max())
                print(ficheiro,text)
        #         output.writerow([ficheiro,i,nuclei_masks.max(),cells_masks.max(),cells_masks.max()/nuclei_masks.max()*100])
        #     output2.writerow(([ficheiro,'NA',nnuclei,ncells,ncells/nnuclei*100]))
                yield [reader[i][chan_cells],reader[i][chan_nuclei],cells_masks,nuclei_masks]
    
        
#     from cellpose import io as iocp
#     from cellpose import plot,utils



    model = cellpose_model(GPU=True)
    model_nuc = cellpose_model(GPU=True,model_type='nuclei')

#     for ficheiro in filelist:
    poseri(filelist,1,0,model,model_nuc)
        
#         average_large_image()
#         result.returned.connect(v.add_image)
#         result.start()


#         worker.start

    #     layers[0].data = []
    #     layers[1].data = []
    #     layers[0] = []
    #     layers[0] = []

** CUDA version installed and working. **
>>>> using GPU
cellpose_residual_on_style_on_concatenation_off
** CUDA version installed and working. **
>>>> using GPU
cellpose_residual_on_style_on_concatenation_off


In [7]:
import time
import numpy as np
import napari

from qtpy.QtCore import Qt
from qtpy.QtWidgets import QPushButton


image = np.random.random((2048, 2048))
zeros = np.zeros((2048, 2048))

@thread_worker
def pan(layer, interval=0.05):
    layer.translate = (-2148, 0)
    layer.data = image
    for i in range(-2140, 8, 16):
        layer.translate = (i, 0)
        yield
        time.sleep(interval)

with napari.gui_qt():
    v = napari.Viewer(show=False)
    v.window._qt_window.setWindowState(Qt.WindowMaximized)
    v.show()

    image_layer = v.add_image(zeros)
    worker = pan(image_layer)

    button = QPushButton("Start animation")
    button.clicked.connect(worker.start)
    v.window.add_dock_widget(widget=button, area='left')

NameError: name 'masks_to_outlines' is not defined

In [30]:
with napari.gui_qt():
    viewer = napari.Viewer()

    @thread_worker(connect={"returned": viewer.add_image})
    def average_large_image():
        return np.random.rand(1024, 512, 512).mean(0)

    average_large_image()

In [7]:
import napari
with napari.gui_qt():
    v = napari.view_image(result[0])

/home/jmamede/anaconda3/lib/python3.7/site-packages/napari/__init__.py:44: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.7. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [ ]:
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

In [ ]:
from cellpose.utils import 

plt.imshow(masks_to_outlines(cells_masks))